In [2]:
import numpy as np

dataset = 'datasets/australian'
dataset = np.genfromtxt("%s.csv" % (dataset), delimiter=",")
X = dataset[:, :-1]
y = dataset[:, -1].astype(int)

In [ ]:
from sklearn.svm import SVC

#Zaimplementowane metody
from StratifiedBagging import StratifiedBagging
from implementedBoosting import implementedBoosting
from RandomSubspace import RandomSubspace
from RandomPatches import RandomPatches

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.base import clone


 clfs = {
     'SVC' : SVC(probability=True, random_state=42),
     'sBag': StratifiedBagging(),    
     'BST': implementedBoosting(base_estimator = SVC(probability=True, random_state=42), n_estimators = 10),
     'RSP' : RandomSubspace(base_estimator = SVC(probability=True, random_state=42), n_estimators = 10, random_state=42),
     'RPT' : RandomPatches(),
 }

In [ ]:
n_splits = 5
n_repeats = 2
rskf = RepeatedStratifiedKFold(
    n_splits=n_splits, n_repeats=n_repeats, random_state=42)
scores = np.zeros((len(clfs), n_splits * n_repeats))

for fold_id, (train, test) in enumerate(rskf.split(X, y)):
    for clf_id, clf_name in enumerate(clfs):
        clf = clone(clfs[clf_name])
        clf.fit(X[train], y[train])
        y_pred = clf.predict(X[test])
        scores[clf_id, fold_id] = accuracy_score(y[test], y_pred)

In [ ]:
mean = np.mean(scores, axis=1)
std = np.std(scores, axis=1)

for clf_id, clf_name in enumerate(clfs):
    print("%s: %.3f (%.2f)" % (clf_name, mean[clf_id], std[clf_id]))

In [ ]:
np.save('results_student', scores)